In [27]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [28]:
# connect pymongo
client = MongoClient("mongodb://localhost:27017")
# client.list_database_names()
# connect Database
db = client["May"]
# connect collection
col = db["0506"]

In [44]:
# 設定要爬取的網址
url = 'https://www.skyscanner.com.tw/transport/flights/tpet/kix/230511/230518/?adults=1&adultsv2=1&cabinclass=economy&children=0&childrenv2=&destinationentityid=27542908&inboundaltsenabled=false&infants=0&originentityid=27547236&outboundaltsenabled=false&preferdirects=false&ref=home&rtn=1'

# 發送 GET 請求並取得 HTML 網頁原始碼
response = requests.get(url)
html = response.text

# 解析 HTML 網頁原始碼
soup = BeautifulSoup(response.text, 'html.parser')

# print(soup.prettify())

In [51]:
import requests
import json
from pymongo import MongoClient

# 設定要爬取的起點和終點城市代碼、出發日和返程日
origin = 'TPE'
destination = 'KIX'
outbound_date = '2022-05-20'
inbound_date = '2022-05-25'

# 設定 API 的 URL 和參數
url = 'https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsedates/v1.0/TW/TWD/zh-TW/{}/{}/{}/{}/'.format(origin, destination, outbound_date, inbound_date)
headers = {
    'X-RapidAPI-Host': 'skyscanner-skyscanner-flight-search-v1.p.rapidapi.com',
    'X-RapidAPI-Key': 'prtl6749387986743898559646983194'
}

# 發送 GET 請求並取得 JSON 資料
response = requests.get(url, headers=headers)
data = response.json()

print(response.text)

{"message":"You are not subscribed to this API."}


In [68]:
import requests
import json
from pymongo import MongoClient

# 設定要爬取的起點和終點城市代碼、出發日和返程日
origin = 'TPE'
destination = 'KIX'
outbound_date = '2022-05-20'
inbound_date = '2022-05-25'

# 設定 API 的 URL 和參數
url = 'https://partners.api.skyscanner.net/apiservices/v3/flights/live/search/create'
headers = {
    'x-api-key': 'prtl6749387986743898559646983194',
    'Content-Type': 'application/json'
}
params = {
    'body': {
        'query': {
            'originPlace': origin,
            'destinationPlace': destination,
            'outboundDate': outbound_date,
            'inboundDate': inbound_date,
            'cabinClass': 'economy',
            'adults': 1,
            'children': 0,
            'infants': 0,
            'locale': 'en-US',
            'currency': 'USD',
            'flightType': 'roundtrip'
        }
    }
}

# 發送 POST 請求並取得 JSON 資料
response = requests.post(url, headers=headers, data=json.dumps(params))
data = response.json()

print(response.text)

{"code":3,"message":"The currency is missing\nThe locale is missing\nThe market is missing\nThe query leg list must contain at least 1 leg\nThe number of adults must be between 1 and 8\nThe cabin class is invalid","details":[]}


In [66]:
# 將 JSON 資料存入 MongoDB
for quote in data['Quotes']:
    doc = {
        'carrier_name': next((carrier['Name'] for carrier in data['Carriers'] if carrier['CarrierId'] == quote['OutboundLeg']['CarrierIds'][0]), None),
        'outbound_date': quote['OutboundLeg']['DepartureDate'],
        'inbound_date': quote['InboundLeg']['DepartureDate'],
        'price': quote['MinPrice']
    }
    col.insert_one(doc)


KeyError: 'Quotes'

In [63]:
print(response.request.__dict__)

{'method': 'POST', 'url': 'https://partners.api.skyscanner.net/apiservices/v3/flights/live/search/create', 'headers': {'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'x-api-key': 'prtl6749387986743898559646983194', 'Content-Type': 'application/json', 'Content-Length': '261'}, '_cookies': <RequestsCookieJar[]>, 'body': '{"body": {"query": {"originPlace": "TPE", "destinationPlace": "KIX", "outboundDate": "2022-05-20", "inboundDate": "2022-05-25", "cabinClass": "economy", "adults": 1, "children": 0, "infants": 0, "locale": "en-US", "currency": "USD", "flightType": "roundtrip"}}}', 'hooks': {'response': []}, '_body_position': None}


In [62]:
print(data)

{'code': 8, 'message': '', 'details': []}


In [56]:
# 連接 MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['May']
collection = db['0506']

In [61]:
# 將 JSON 資料存入 MongoDB
for quote in data['Quotes']:
    doc = {
        'carrier_name': next((carrier['Name'] for carrier in data['Carriers'] if carrier['CarrierId'] == quote['OutboundLeg']['CarrierIds'][0]), None),
        'outbound_date': quote['OutboundLeg']['DepartureDate'],
        'inbound_date': quote['InboundLeg']['DepartureDate'],
        'price': quote['MinPrice']
    }
    collection.insert_one(doc)

print('Done!')

KeyError: 'Quotes'